In [59]:
# 이미지 여백만들고 회전시키기 (원본)

import os
import glob
import cv2

# 경로 설정
# input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
# output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

input_path = 'D:/coding/data_set/lomin/train_copy3/temp1/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp2/'

# 경로 폴더 만들기
try :
    os.makedirs(output_path)
except:
    pass


img_list = sorted(glob.glob(input_path + '/*'))

for i in img_list:
    if not '.ini' in i:
        # print(i.split('\\')[1].split('.')[0])
        src = cv2.imread(i, cv2.IMREAD_COLOR)
        height, width, channel = src.shape
        my_angle = 45

        ##                                  중심점(center), 각도(angle), 비율(scale)
        matrix = cv2.getRotationMatrix2D((width/2, height/2), -my_angle, 1)
        # 비율 : 가로길이 / 대각선

        #                                           흰색 : borderValue=(255, 255, 255)
        dst = cv2.warpAffine(src, matrix, (width, height), borderValue=(255, 255, 255))

        # 변경 이미지 저장 (파일명_각도)
        cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)



In [41]:
# 이미지 여백만들고 회전시키기 (원본)

import os
import glob
import cv2

# 경로 설정
# input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
# output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

input_path = 'D:/coding/data_set/lomin/train_copy3/temp1/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp2/'

def ro_img_lite(angle, input_path, output_path):
    # 경로 폴더 만들기
    try :
        os.makedirs(output_path)
    except:
        pass

    img_list = sorted(glob.glob(input_path + '/*'))

    for i in img_list:
        if not '.ini' in i:
            # print(i.split('\\')[1].split('.')[0])
            src = cv2.imread(i, cv2.IMREAD_COLOR)
            height, width, channel = src.shape
            my_angle = angle

            ##                                  중심점(center), 각도(angle), 비율(scale)
            matrix = cv2.getRotationMatrix2D((width/2, height/2), -my_angle, 1)
            # 비율 : 가로길이 / 대각선

            #                                           흰색 : borderValue=(255, 255, 255)
            dst = cv2.warpAffine(src, matrix, (width, height), borderValue=(255, 255, 255))

            # 변경 이미지 저장 (파일명_각도)
            cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)
            # cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '.png', dst)



In [37]:
## 이미지 크롭 (500*500)

import os
import glob
import cv2
import random

# 경로 설정
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'


img_list = sorted(glob.glob(input_path + '/*'))

for i in img_list:

    src = cv2.imread(i, cv2.IMREAD_COLOR)
    
    height, width, channel = src.shape
    
    ## 이미지 에서 500*500 이미지를 crop 함
    x_range = width - 501
    y_range = height - 501
    
    for j in range(0, 3):
        x1 = x_random = random.randint(0, x_range)
        y1 = y_random = random.randint(0, y_range)
        x2 = x1 + 500
        y2 = y1 + 500 
        print(y1, y2, x1, x2)
        
        dst = src[y1:y2, x1:x2]

        # cv2.imshow("src", src)
        # cv2.imshow("dst", dst)
        # cv2.waitKey()
        # cv2.destroyAllWindows()

        cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '(' + str(j) + ')' + '.png', dst)
        
        # 문제점 - 회전을 한 이미지 crop 시 좌표도 회전하게 됨. (이미지 짤림)
        # affine 은 이미지를 회전, 왜곡 해도 좌표 정보를 그대로 가지고 있음.

1580 2080 744 1244
295 795 1105 1605
1434 1934 259 759


In [22]:
## 이미지 크롭 (500*500)

import os
import glob
import cv2
import random

# 경로 설정
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

def crop_500(my_angle, input_path, output_path):
    img_list = sorted(glob.glob(input_path + '/*'))

    for i in img_list:

        src = cv2.imread(i, cv2.IMREAD_COLOR)
        
        height, width, channel = src.shape
        
        ## 이미지 에서 500*500 이미지를 crop 함
        x_range = width - 501
        y_range = height - 501
        
        for j in range(0, 3):
            x1 = x_random = random.randint(0, x_range)
            y1 = y_random = random.randint(0, y_range)
            x2 = x1 + 500
            y2 = y1 + 500 
            # print(y1, y2, x1, x2)
            
            dst = src[y1:y2, x1:x2]
            
            cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '(' + str(j) + ')' + '.png', dst)
            
            # 문제점 - 회전을 한 이미지 crop 시 좌표도 회전하게 됨. (이미지 짤림)
            # affine 은 이미지를 회전, 왜곡 해도 좌표 정보를 그대로 가지고 있음.

---

In [17]:
# 이미지 크롭
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

crop_500(input_path, output_path)

In [20]:
# 이미지 회전
input_path = 'D:/coding/data_set/lomin/train_copy3/temp1/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

ro_img_lite(55, input_path, output_path)

---

In [ ]:
# 이미지 여백만들고 회전시키기 (원본)

import os
import glob
import cv2

# 경로 설정
# input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
# output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

input_path = 'D:/coding/data_set/lomin/train_copy3/temp1/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp2/'

def ro_img_lite(angle, input_path, output_path):
    # 경로 폴더 만들기
    try :
        os.makedirs(output_path)
    except:
        pass

    img_list = sorted(glob.glob(input_path + '/*'))

    for i in img_list:
        if not '.ini' in i:
            # print(i.split('\\')[1].split('.')[0])
            src = cv2.imread(i, cv2.IMREAD_COLOR)
            height, width, channel = src.shape
            my_angle = angle

            ##                                  중심점(center), 각도(angle), 비율(scale)
            matrix = cv2.getRotationMatrix2D((width/2, height/2), -my_angle, 1)
            # 비율 : 가로길이 / 대각선

            #                                           흰색 : borderValue=(255, 255, 255)
            dst = cv2.warpAffine(src, matrix, (width, height), borderValue=(255, 255, 255))

            # 변경 이미지 저장 (파일명_각도)
            # cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)
            cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '.png', dst)



In [43]:
## 이미지 크롭 (500*500)

import os
import glob
import cv2
import random
import numpy as pd

# 경로 설정
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

# def img_crop_ro(my_angle, input_path, output_path):
my_angle = 45
img_list = sorted(glob.glob(input_path + '/*'))

for i in img_list:

    src = cv2.imread(i, cv2.IMREAD_COLOR)
    
    height, width, channel = src.shape
    
    ## 이미지 에서 500*500 이미지를 crop 함
    x_range = width - 501
    y_range = height - 501
    
    for j in range(0, 3):
        x1 = x_random = random.randint(0, x_range)
        y1 = y_random = random.randint(0, y_range)
        x2 = x1 + 500
        y2 = y1 + 500 
        print(y1, y2, x1, x2)
        
        dst = src[y1:y2, x1:x2]


        cv2.imwrite(output_path + i.split('\\')[1].split('.')[0]  + '(' + str(j) + ')' + '_' + str(my_angle) + '.png', dst)
        

    ro_img_lite(my_angle, output_path, output_path)
        # 문제점 - 회전을 한 이미지 crop 시 좌표도 회전하게 됨. (이미지 짤림)
        # affine 은 이미지를 회전, 왜곡 해도 좌표 정보를 그대로 가지고 있음.

In [49]:
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

img_crop_ro(66, input_path, output_path)

440 940 917 1417
1733 2233 313 813
35 535 301 801
187 687 243 743
1193 1693 1003 1503
770 1270 914 1414
411 911 789 1289
1205 1705 827 1327
226 726 441 941
66 566 518 1018
389 889 736 1236
731 1231 995 1495


---

In [103]:
def rotation_crop(angle, path):
    # print(i.split('\\')[1].split('.')[0])
    src = cv2.imread(path, cv2.IMREAD_COLOR)
    height, width, channel = src.shape
    my_angle = angle

    ##                                  중심점(center), 각도(angle), 비율(scale)
    matrix = cv2.getRotationMatrix2D((width/2, height/2), -my_angle, 1)
    # 비율 : 가로길이 / 대각선

    #                                           흰색 : borderValue=(255, 255, 255)
    dst = cv2.warpAffine(src, matrix, (width, height), borderValue=(255, 255, 255))

    # 변경 이미지 저장 (파일명_각도)
    # cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)
    cv2.imwrite(path, dst)

In [107]:
import os
import glob
import cv2
import random
import numpy as np

def lotation_img(img_count, input_path, output_path):
    # 폴더 만들기
    try :
        os.makedirs(output_path)
    except:
        pass

    img_list = sorted(glob.glob(input_path + '/*'))

    # 각도
    for j in range(0, 360):
        # 전체 이미지 중 랜덤하게 추출
        random_list = np.random.choice(img_list, img_count, replace=False)
        
        for i in random_list:
            src = cv2.imread(i, cv2.IMREAD_COLOR)
            
            height, width, channel = src.shape
            
            ## 이미지 에서 500*500 이미지를 crop 함
            x_range = width - 501
            y_range = height - 501
            
            for k in range(0, 3):
                x1 = x_random = random.randint(0, x_range)
                y1 = y_random = random.randint(0, y_range)
                x2 = x1 + 500
                y2 = y1 + 500 
                # print(y1, y2, x1, x2)
                
                dst = src[y1:y2, x1:x2]


                total_path = output_path + i.split('\\')[-1].split('.')[0] + '('+str(k)+")_" + str(j) + '.png' 
                cv2.imwrite(total_path, dst)
                
                # 이미지 회전
                rotation_crop(j, total_path)
            


In [ ]:
input_path = 'D:/coding/data_set/lomin/train_copy3/temp/' 
output_path = 'D:/coding/data_set/lomin/train_copy3/temp1/'

lotation_img(30, input_path, output_path)

In [ ]:
## 설정 회전각 1개의 각에 의해 저장
import os
import glob
import cv2

input_path = 'D:/coding/data_set/lomin/train_copy2/train' 
output_path = 'D:/coding/data_set/lomin/train_copy2/train_a/'

img_list = sorted(glob.glob(input_path + '/*'))

# 설정 각도에 따라 저장.
def lotation_img(my_angle):
    for i in img_list:
        # print(i.split('\\')[1].split('.')[0])
        src = cv2.imread(i, cv2.IMREAD_COLOR)
        height, width, channel = src.shape
        
        ##                                  중심점(center), 각도(angle), 비율(scale)
        matrix = cv2.getRotationMatrix2D((width/2, height/2), my_angle, 0.57)
        # 비율 : 가로길이 / 대각선

        dst = cv2.warpAffine(src, matrix, (width, height))

        # 변경 이미지 저장 (파일명_각도)
        # cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)
        
        cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '.png', dst)

---

In [ ]:
# Annotation 새로 만들기
#https://junyoung-jamong.github.io/machine/learning/2019/01/23/%EB%B0%94%EC%9A%B4%EB%94%A9%EB%B0%95%EC%8A%A4%EB%A5%BC-%ED%8F%AC%ED%95%A8%ED%95%9C-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EC%A6%9D%ED%8F%AD%EC%8B%9C%ED%82%A4%EA%B8%B0-with-imgaug.html

# 바운딩박스 복붙 (Annotation정보 읽기)
import os
import glob
import cv2
import xml.etree.ElementTree as ET

# Annotation 반복 만들기 (기본 0도)
def annotation_xml(origin_path):
    
    # 폴더 내 이미지 리스트
    img_list = sorted(glob.glob(origin_path + '/*'))

    for i in img_list:
        File = open(origin_path + i.split('\\')[1].split('.')[0] + ".xml", "w")
        print("""<annotation>
    <folder>test_after</folder>
    <filename>{}.png</filename>
    <path>D:\coding\data_set\lomin\\train_copy\\train\\{}.png</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>1654</width>
        <height>2339</height>
        <depth>1</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>0</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>355</xmin>
            <ymin>503</ymin>
            <xmax>1299</xmax>
            <ymax>1836</ymax>
        </bndbox>
    </object>
</annotation>
        """.format(i.split('\\')[1].split('.')[0], i.split('\\')[1].split('.')[0]), file=File)

        File.close()

In [ ]:
%%time
## 이미지와 xml 파일을 랜덤으로 골라 각도별 폴더에 복사 붙여넣기 (memory out 방지)
## 주의!! annotation 은 회전된 데이터가 아님
## mtrixdml 각도를 0으로 고정하면, 배경만 추가한 데이터 생성
import glob
import numpy as np
import cv2
import shutil

# dir = 'D:/coding/data_set/lomin/train_copy2/train/'

dir = 'D:/coding/data_set/lomin/train_copy2/test/'

# # 새 파일 넣을 경로 및 폴더 만들기 (train)
# dir_after = 'D:/coding/data_set/lomin/train_copy/train_angle/'
# os.makedirs(dir_after)

# 새 파일 넣을 경로 및 폴더 만들기 (val)
dir_after = 'D:/coding/data_set/lomin/train_copy2/test_angle/'
os.makedirs(dir_after)

# dir 에 png 파일 선택 (내림차순)
dir_png = sorted(glob.glob(dir+'/*.png'))

## 각 폴더마다 몇개의 샘플을 만들 것인가?
num_img = 30


# 원하는 각도 만큼 폴더 생성
for i in range(0,360):
    
    # dir의 png 파일 중 랜덤으로 n개 선택 (비중복)
    png_random = sorted(np.random.choice(dir_png, num_img, replace=False))
    
    my_angle = i

    # 각도별 폴더 생성
    os.makedirs(dir_after + str(my_angle)) 
    
    for idx in png_random:
        src = cv2.imread(idx, cv2.IMREAD_COLOR)
        height, width, channel = src.shape
        ##                                  중심점(center), 각도(angle), 비율(scale)
        matrix = cv2.getRotationMatrix2D((width/2, height/2), 0, 0.57)
        # matrix = cv2.getRotationMatrix2D((width/2, height/2), my_angle, 0.57)
        dst = cv2.warpAffine(src, matrix, (width, height))

        # 이미지 파일 저장 경로
        png_random = idx.split('\\')[1].split('.')[0] + '.png'
        cv2.imwrite(dir_after + str(my_angle) + '/' + png_random, dst)
            
        # 랜덤 이미지와 같은 xml 파일을 복사
        xml_random = idx.split('\\')[0]+'/'+idx.split('\\')[-1].split('.')[0]+'.xml'
        shutil.copy(xml_random, dir_after + str(my_angle) + '/')


---

In [ ]:
# from def_all

%%time
####### 폴더 추가해서 복사하기 ####################

## 테두리가 만들어진 이미지 폴더들을 순화하며 폴더 명에 따라 회전 시키기

## 순환이 문자로(j) 인식되기 때문에 0, 1, 10, 100, 101, 102, 103... 109, 11, 110.. 순으로 작업 수행

import imgaug as ia
import os
from imgaug import augmenters as iaa
# from files import *
from pascal_voc_writer import Writer

ia.seed(1)

# dir = 'D:/coding/data_set/lomin/train_copy/train_angle/' # train 테스트셋
# dir = 'D:/coding/data_set/lomin/train_copy/val_angle/' # val 테스트셋

dir = 'D:/coding/data_set/lomin/temp/val_angle/' # val 테스트셋

# 모든 폴더 순환
for j in sorted(os.listdir(dir)):
    dir_d = dir + j + '/' 

    # os.makedirs(dir_d + '/dgr_' + j) 
    dir_2 = dir_dgr = dir_d + '/dgr_' + j + '/'
    

    images, annotations = read_train_dataset(dir_d)

    for idx in range(len(images)):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.Sequential([
            # iaa.Multiply((1.2, 1.5)),
            iaa.Affine(
                # translate_px={"x": 40, "y": 60},
                # scale=(0.5, 0.7),
                rotate = int(j)
            )
        ])

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        # new_image_file = dir + 'after_' + annotations[idx][2]
        # new_image_file = dir_d + annotations[idx][2]
        new_image_file = dir_dgr + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            # name (label) , x1, x2, y1, y2 값 바꿈
            voc_writer.addObject(j), int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        # voc_writer.save(dir + 'after_' + annotations[idx][1])
        # voc_writer.save(dir_d + annotations[idx][1])
        voc_writer.save(dir_dgr + annotations[idx][1])


In [ ]:
%%time
####### 폴더 추가해서 복사하기 ####################

## 테두리가 만들어진 이미지 폴더들을 순화하며 폴더 명에 따라 회전 시키기

## 순환이 문자로(j) 인식되기 때문에 0, 1, 10, 100, 101, 102, 103... 109, 11, 110.. 순으로 작업 수행

import imgaug as ia
import os
from imgaug import augmenters as iaa
# from files import *
from pascal_voc_writer import Writer

ia.seed(1)

# # 새 파일 넣을 경로 및 폴더 만들기 (train)
# dir_after_ro = 'D:/coding/data_set/lomin/train_copy2/train_angle_ro/'
# os.makedirs(dir_after_ro)

# 새 파일 넣을 경로 및 폴더 만들기 (val)
dir_after_ro = 'D:/coding/data_set/lomin/train_copy2/test_angle_ro/'
os.makedirs(dir_after_ro)


# 파일 읽을 경로
# dir = 'D:/coding/data_set/lomin/train_copy2/train_angle/' # train 테스트셋
dir = 'D:/coding/data_set/lomin/train_copy2/test_angle/' # test 테스트셋

# 모든 폴더 순환
for j in sorted(os.listdir(dir)):
    dir_d = dir + j + '/' 

    os.makedirs(dir_after_ro + '/dgr_' + j) 

    # 각 폴더에 이미지 저장
    dir_dgr = dir_after_ro + '/dgr_' + j + '/'

    # 한 폴더에 이미지 모두 저장
    # dir_dgr = dir_after_ro
    

    images, annotations = read_train_dataset(dir_d)

    for idx in range(len(images)):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.Sequential([
            # iaa.Multiply((1.2, 1.5)),
            iaa.Affine(
                # translate_px={"x": 40, "y": 60},
                # scale=(0.5, 0.7),
                rotate = int(j)
            )
        ])

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        # new_image_file = dir + 'after_' + annotations[idx][2]
        # new_image_file = dir_d + annotations[idx][2]
        # new_image_file = dir_dgr + annotations[idx][2]
        new_image_file = dir_dgr + annotations[idx][2].split('.')[0] + '_' + j + "." + annotations[idx][2].split('.')[1]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            # name (label) , x1, x2, y1, y2 값 바꿈
            voc_writer.addObject(j, int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        # voc_writer.save(dir + 'after_' + annotations[idx][1])
        # voc_writer.save(dir_d + annotations[idx][1])
        # voc_writer.save(dir_dgr + annotations[idx][1])
        voc_writer.save(dir_dgr + annotations[idx][1].split('.')[0] + '_' + j + "." + annotations[idx][1].split('.')[1])


In [ ]:
# from project4-82(제출) copy 
# 통합 폴더에 파일 있을 때, (나눠 저장 하지 않을때 사용)

%%time
## 테이터 프레임에 xml 정보 추가 (통합 폴더)

import imgaug as ia
import os
from imgaug import augmenters as iaa
import xml.etree.ElementTree as ET

ia.seed(1)

# train dir
dir = DATA_PATH + '/train_angle_ro/'

# val dir
# dir = DATA_PATH + '/val_angle_ro/'


# # 모든 폴더 순환
# for j in sorted(os.listdir(dir)):

#         if not '.ini' in j:
#             dir_2 = dir + j + '/'
     
## 폴더안에 있는 모든 xml 파일 이름 가져오기
file_list = os.listdir(dir)
xml_list = []

for file in file_list:
    if '.xml' in file:
        xml_list.append(file)

## 모든 .xml파일에 대해 수정
for xml_file in xml_list:
    target_path = dir + xml_file

    tree = ET.parse(target_path)
    root = tree.getroot()
    bounding_box_list = []

    file_name = root.find('filename').text
    object_label = root.find('object/name').text
    x_min = int(root.find('object/bndbox/xmin').text)
    y_min = int(root.find('object/bndbox/ymin').text)
    x_max = int(root.find('object/bndbox/xmax').text)
    y_max = int(root.find('object/bndbox/ymax').text)
            
    bounding_box = [file_name, x_min, y_min, x_max, y_max, object_label]

    # trian 데이터 프레임에 추가하기
    df_train = df_train.append(pd.DataFrame([[file_name, x_min, y_min, x_max, y_max, object_label]], 
                                            columns=['img_file', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']),
                                            ignore_index=True)
    
    # # val 데이터 프레임에 추가하기
    # df_val = df_val.append(pd.DataFrame([[file_name, x_min, y_min, x_max, y_max, object_label]], 
    #                                         columns=['img_file', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']),
    #                                         ignore_index=True)



In [ ]:
## 이미지 데이터 읽기

from os import listdir
import cv2
import numpy as np

def read_train_dataset(dir):
    images = []
    annotations = []

    for file in listdir(dir):
        if 'jpg' in file.lower() or 'png' in file.lower():
            images.append(cv2.imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_anntation(dir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))

    # images = np.array(images)

    return images, annotations

In [ ]:
# from def_all

# 이미지 여백만들고 회전시키기 (원본)

import os
import glob
import cv2

test_path = tr_path # train

img_list = sorted(glob.glob(test_path + '/*'))

for i in img_list:
    # print(i.split('\\')[1].split('.')[0])
    src = cv2.imread(i, cv2.IMREAD_COLOR)
    height, width, channel = src.shape
    my_angle = 0

    ##                                  중심점(center), 각도(angle), 비율(scale)
    matrix = cv2.getRotationMatrix2D((width/2, height/2), my_angle, 0.57)
    # 비율 : 가로길이 / 대각선

    dst = cv2.warpAffine(src, matrix, (width, height))

    # 변경 이미지 저장 (파일명_각도)
    cv2.imwrite('D:/coding/data_set/lomin/train_copy2/train_after/' + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)



In [ ]:
%%time
## 이미 0으로 적용된 name 을 각도 별로 다시 지정 (1회용)

import imgaug as ia
import glob
import os
from imgaug import augmenters as iaa
# from files import *
from pascal_voc_writer import Writer

import xml.etree.ElementTree as ET

ia.seed(1)

# dir = 'D:/coding/data_set/lomin/train_copy/train_angle/' # train 테스트셋
# dir = 'D:/coding/data_set/lomin/train_copy/test_angle/' # test 테스트셋

dir = 'D:/coding/data_set/lomin/temp/test_angle/' # test 테스트셋

# 모든 폴더 순환
for j in sorted(os.listdir(dir)):
    dir_d = dir + j + '/' 

    # os.makedirs(dir_d + '/dgr_' + j) 
    dir_2 = dir_dgr = dir_d + 'dgr_' + j #+ '/'
    
    # ---
    # f = dir_2
    # label = f.split('/')[-2].split('_')[1]
    
    # print(label)
    # # print(os.listdir(f))
    # dir_png = sorted(glob.glob(dir_2+'/*.xml'))
    # print(dir_png)
    
    # # for i in os.listdir(f):
    # ---
    

    ## 폴더안에 있는 모든 xml 파일 이름 가져오기
    file_list = os.listdir(dir_2)
    xml_list = []
    
    for file in file_list:
        if '.xml' in file:
            xml_list.append(file)

    ## 모든 .xml파일에 대해 수정
    for xml_file in xml_list:
        target_path = dir_2 + "/" + xml_file
        targetXML = open(target_path, 'rt', encoding='UTF8')

        tree = ET.parse(targetXML)

        root = tree.getroot()
        
        ##수정할 부분
        target_tag = root.find("object/name")  # boject 하위 name 

        original = target_tag.text     #원본 String
        modified = original.replace(r"0",j)  # 0을 j 로 바꿈
        # modified = modified.replace("/", "\\")
        target_tag.text = modified  #수정
        
        tree.write(target_path)
        
## https://skkim1080.tistory.com/entry/Python%EC%9C%BC%EB%A1%9C-xml-%ED%8C%8C%EC%9D%BC-%EB%82%B4%EC%9A%A9-%EC%88%98%EC%A0%95%ED%95%98%EA%B8%B0



In [ ]:
import pandas as pd

# train 데이터 프레임 만들기
df_train = pd.DataFrame(columns=['img_file', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class'])
df_test = pd.DataFrame(columns=['img_file', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class'])

In [ ]:
%%time
## 테이터 프레임에 xml 정보 추가 (각각 폴더 순환)

import imgaug as ia
import os
from imgaug import augmenters as iaa
import xml.etree.ElementTree as ET

ia.seed(1)

# train dir
dir = DATA_PATH + '/train_angle_ro/'

# val dir
# dir = DATA_PATH + '/val_angle_ro/'


# 모든 폴더 순환
for j in sorted(os.listdir(dir)):

        if not '.ini' in j:           
            dir_2 = dir + j + '/'
     
            ## 폴더안에 있는 모든 xml 파일 이름 가져오기
            file_list = os.listdir(dir_2)
            xml_list = []
            
            for file in file_list:
                if '.xml' in file:
                    xml_list.append(file)

            ## 모든 .xml파일에 대해 수정
            for xml_file in xml_list:
                target_path = dir_2 + xml_file

                tree = ET.parse(target_path)
                root = tree.getroot()
                bounding_box_list = []

                file_name = root.find('filename').text
                object_label = root.find('object/name').text
                x_min = int(root.find('object/bndbox/xmin').text)
                y_min = int(root.find('object/bndbox/ymin').text)
                x_max = int(root.find('object/bndbox/xmax').text)
                y_max = int(root.find('object/bndbox/ymax').text)
                        
                bounding_box = [file_name, x_min, y_min, x_max, y_max, object_label]

                # trian 데이터 프레임에 추가하기
                df_train = df_train.append(pd.DataFrame([[file_name, x_min, y_min, x_max, y_max, object_label]], 
                                                        columns=['img_file', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']),
                                                        ignore_index=True)
                
                # # val 데이터 프레임에 추가하기
                # df_test = df_test.append(pd.DataFrame([[file_name, x_min, y_min, x_max, y_max, object_label]], 
                #                                         columns=['img_file', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']),
                #                                         ignore_index=True)



In [ ]:
%%time
## 테이터 프레임에 xml 정보 추가 (통합 폴더)

import imgaug as ia
import os
from imgaug import augmenters as iaa
import xml.etree.ElementTree as ET

ia.seed(1)

# train dir
# dir = DATA_PATH + '/train_angle_ro/'

# val dir
dir = DATA_PATH + '/val_angle_ro/'

     
## 폴더안에 있는 모든 xml 파일 이름 가져오기
file_list = os.listdir(dir)
xml_list = []

for file in file_list:
    if '.xml' in file:
        xml_list.append(file)

## 모든 .xml파일에 대해 수정
for xml_file in xml_list:
    target_path = dir + xml_file

    tree = ET.parse(target_path)
    root = tree.getroot()
    bounding_box_list = []

    file_name = root.find('filename').text
    object_label = root.find('object/name').text
    x_min = int(root.find('object/bndbox/xmin').text)
    y_min = int(root.find('object/bndbox/ymin').text)
    x_max = int(root.find('object/bndbox/xmax').text)
    y_max = int(root.find('object/bndbox/ymax').text)
            
    bounding_box = [file_name, x_min, y_min, x_max, y_max, object_label]

    # # trian 데이터 프레임에 추가하기
    # df_train = df_train.append(pd.DataFrame([[file_name, x_min, y_min, x_max, y_max, object_label]], 
    #                                         columns=['img_file', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']),
    #                                         ignore_index=True)
    
    # val 데이터 프레임에 추가하기
    df_test = df_test.append(pd.DataFrame([[file_name, x_min, y_min, x_max, y_max, object_label]], 
                                            columns=['img_file', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']),
                                            ignore_index=True)



In [ ]:

## 테이터 프레임에 xml 정보 추가 (통합 폴더)

import imgaug as ia
import os
from imgaug import augmenters as iaa
import xml.etree.ElementTree as ET

def xml_to_df(dir, df_set):
    
    ia.seed(1)
        
    ## 폴더안에 있는 모든 xml 파일 이름 가져오기
    file_list = os.listdir(dir)
    xml_list = []

    for file in file_list:
        if '.xml' in file:
            xml_list.append(file)

    ## 모든 .xml파일에 대해 수정
    for xml_file in xml_list:
        target_path = dir + xml_file

        tree = ET.parse(target_path)
        root = tree.getroot()

        file_name = root.find('filename').text
        object_label = root.find('object/name').text
        x_min = int(root.find('object/bndbox/xmin').text)
        y_min = int(root.find('object/bndbox/ymin').text)
        x_max = int(root.find('object/bndbox/xmax').text)
        y_max = int(root.find('object/bndbox/ymax').text)
                
        # trian 데이터 프레임에 추가하기
        df_set = df_set.append(pd.DataFrame([[file_name, x_min, y_min, x_max, y_max, object_label]], 
                                                columns=['img_file', 'bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']),
                                                ignore_index=True)



In [ ]:
%%time
# train 
dir = DATA_PATH + '/train_angle/'
xml_to_df(dir, df_train)


Wall time: 16.8 s


In [ ]:
%%time
# test 
dir = DATA_PATH + '/test_angle/'
df_set = df_test
xml_to_df(dir, df_set)

Wall time: 5.79 s
